In [15]:
import os 
import sys

'''
Create directory of dataset:
- efs_dir: mount point of efs
- dataset_dir: name of a directory to store dataset
'''
efs_dir = "/home/ec2-user/SageMaker/efs"
dataset_dir = "food101"
if not os.path.exists(efs_dir):
    print("Mount EFS on notebook instance. Food-101 dataset is very large.")
    sys.exit()
elif not os.path.exists(os.path.join(efs_dir, dataset_dir)):
    print("Creating dataset directory {}".format(os.path.join(efs_dir, dataset_dir)))
    os.mkdir(os.path.join(efs_dir, dataset_dir))
    
'''
Download dataset (This takes 20-30 minutes.)
'''
import urllib.request
url = 'http://data.vision.ee.ethz.ch/cvl/food-101.tar.gz'
urllib.request.urlretrieve(url, os.path.join(efs_dir, dataset_dir, 'food-101.tar.gz'))

('/home/ec2-user/SageMaker/efs/food101/food-101.tar.gz',
 <http.client.HTTPMessage at 0x7f5ad412d128>)

In [ ]:
import tarfile
file_name =  os.path.join(efs_dir, dataset_dir, 'food-101.tar.gz')
tar = tarfile.open(file_name, "r:gz")
tar.extractall(path=os.path.join(efs_dir, dataset_dir))
tar.close()

In [1]:
import os
efs_dir = "/home/ec2-user/SageMaker/efs"
dataset_dir = "food101"
food_name = 'ramen'
edge_dir = 'ramen_edge'
image_path = os.path.join(efs_dir, dataset_dir, 'food-101', 'images', food_name)
edge_path = os.path.join(efs_dir, dataset_dir,'food-101', 'images', edge_dir)

from PIL import Image, ImageFilter
if not os.path.exists(edge_path):
    print("Create directory to store edge-images")
    os.mkdir(edge_path)

count = 0
for f in os.listdir(image_path):
    print('\r extracting edge from images {}/{}'.format(str(count+1),len(os.listdir(image_path))), end="")
    image = Image.open(os.path.join(image_path, f))
    image = image.filter(ImageFilter.FIND_EDGES)
    image = image.filter(ImageFilter.CONTOUR)
    image = image.convert("L")           
    image = image.point(lambda x: 0 if x < 100 else x)
    image.save(os.path.join(edge_path, f)) 
    count+=1

 extracting edge from images 1000/1000

In [2]:
import sagemaker
from sagemaker.mxnet import MXNet

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

input_img = sagemaker_session.upload_data(path=edge_path, key_prefix='food101/edge_ramen')
output_img  = sagemaker_session.upload_data(path=image_path, key_prefix='food101/image_ramen')

In [3]:
!wget https://raw.githubusercontent.com/awslabs/amazon-sagemaker-examples/master/sagemaker-python-sdk/mxnet_gluon_cifar10/setup.sh
!sh ./setup.sh

--2018-09-16 02:55:47--  https://raw.githubusercontent.com/awslabs/amazon-sagemaker-examples/master/sagemaker-python-sdk/mxnet_gluon_cifar10/setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.88.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.88.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2445 (2.4K) [text/plain]
Saving to: ‘setup.sh.1’

setup.sh.1          100%[===================>]   2.39K  --.-KB/s    in 0s      

2018-09-16 02:55:47 (46.0 MB/s) - ‘setup.sh.1’ saved [2445/2445]

Loaded plugins: dkms-build-requires, priorities, update-motd, upgrade-helper
Resolving Dependencies
--> Running transaction check
---> Package docker.x86_64 0:17.12.1ce-1.135.amzn1 will be erased
--> Finished Dependency Resolution

Dependencies Resolved

 Package     Arch        Version                       Repository          Size
Removing:
 docker      x86_64      17.12.1ce-1.135.amzn1         @amzn-updates 

--> Finished Dependency Resolution

Dependencies Resolved

 Package                   Arch   Version        Repository                Size
Installing:
 nvidia-docker2            noarch 2.0.3-1.docker17.09.1.ce.amzn1
                                                 nvidia-docker            4.3 k
Installing for dependencies:
 libnvidia-container-tools x86_64 1.0.0-0.1.rc.2 libnvidia-container       34 k
 libnvidia-container1      x86_64 1.0.0-0.1.rc.2 libnvidia-container       80 k
 nvidia-container-runtime  x86_64 2.0.0-1.docker17.09.1.amzn1
                                                 nvidia-container-runtime 2.1 M
 nvidia-container-runtime-hook
                           x86_64 1.4.0-1.amzn1  nvidia-container-runtime 706 k

Transaction Summary
Install  1 Package (+4 Dependent packages)

Total download size: 2.9 M
Installed size: 8.2 M
(1/5): libnvidia-container-tools-1.0.0-0.1.rc.2.x86_64.rpm |  34 kB   00:00     
(2/5): nvidia-docker2-2.0.3-1.docker17.09.1.ce.amzn1.noarc | 4.3 kB

In [10]:
mxnet_estimator = MXNet('pix2pix.py',
                          role=role, 
                          train_instance_count=1,
                          train_instance_type='ml.p3.2xlarge',
                          hyperparameters={'batch_size': 32,
                                           'epochs': 1000,
                                           'learning_rate': 0.0002,
                                           'beta1': 0.5,
                                           'lambda1': 100})

In [ ]:
mxnet_estimator.fit({'feature': input_img, 'label': output_img})

INFO:sagemaker:Creating training-job with name: sagemaker-mxnet-2018-09-16-03-31-05-846


...........................
2018-09-16 03:35:22,750 INFO - root - running container entrypoint
2018-09-16 03:35:22,750 INFO - root - starting train task
2018-09-16 03:35:22,770 INFO - container_support.training - Training starting
2018-09-16 03:35:23,881 INFO - mxnet_container.train - MXNetTrainingEnvironment: {'enable_cloudwatch_metrics': False, 'available_gpus': 1, 'channels': {u'feature': {u'TrainingInputMode': u'File', u'RecordWrapperType': u'None', u'S3DistributionType': u'FullyReplicated'}, u'label': {u'TrainingInputMode': u'File', u'RecordWrapperType': u'None', u'S3DistributionType': u'FullyReplicated'}}, '_ps_verbose': 0, 'resource_config': {u'hosts': [u'algo-1'], u'network_interface_name': u'ethwe', u'current_host': u'algo-1'}, 'user_script_name': u'pix2pix.py', 'input_config_dir': '/opt/ml/input/config', 'channel_dirs': {u'feature': u'/opt/ml/input/data/feature', u'label': u'/opt/ml/input/data/label'}, 'code_dir': '/opt/ml/code', 'output_data_dir': '/opt/ml/output/data/', 'ou

2018-09-16 03:36:15,458:304:INFO:speed: 92.2349577919 samples/s
2018-09-16 03:36:15,458 INFO - LoggingTest - speed: 92.2349577919 samples/s
2018-09-16 03:36:15,667:307:INFO:discriminator loss = 0.234343, generator loss = 46.043285, binary training acc = 0.857897 at iter 30 epoch 2
2018-09-16 03:36:15,667 INFO - LoggingTest - discriminator loss = 0.234343, generator loss = 46.043285, binary training acc = 0.857897 at iter 30 epoch 2
2018-09-16 03:36:15,820:313:INFO:
binary training acc at epoch 2: facc=0.860942
2018-09-16 03:36:15,820 INFO - LoggingTest - 
binary training acc at epoch 2: facc=0.860942
2018-09-16 03:36:15,820:314:INFO:time: 11.157791
2018-09-16 03:36:15,820 INFO - LoggingTest - time: 11.157791
2018-09-16 03:36:16,166:304:INFO:speed: 92.4788697846 samples/s
2018-09-16 03:36:16,166 INFO - LoggingTest - speed: 92.4788697846 samples/s
2018-09-16 03:36:16,377:307:INFO:discriminator loss = 0.597344, generator loss = 40.514374, binary training acc = 0.820469 at iter 0 epoch 3
2

2018-09-16 03:36:56,587:304:INFO:speed: 91.6343358071 samples/s
2018-09-16 03:36:56,587 INFO - LoggingTest - speed: 91.6343358071 samples/s
2018-09-16 03:36:56,796:307:INFO:discriminator loss = 0.111938, generator loss = 34.133911, binary training acc = 0.910916 at iter 20 epoch 6
2018-09-16 03:36:56,796 INFO - LoggingTest - discriminator loss = 0.111938, generator loss = 34.133911, binary training acc = 0.910916 at iter 20 epoch 6
2018-09-16 03:37:00,070:304:INFO:speed: 91.326818368 samples/s
2018-09-16 03:37:00,070 INFO - LoggingTest - speed: 91.326818368 samples/s
2018-09-16 03:37:00,282:307:INFO:discriminator loss = 0.469618, generator loss = 35.075073, binary training acc = 0.917737 at iter 30 epoch 6
2018-09-16 03:37:00,282 INFO - LoggingTest - discriminator loss = 0.469618, generator loss = 35.075073, binary training acc = 0.917737 at iter 30 epoch 6
2018-09-16 03:37:00,428:313:INFO:
binary training acc at epoch 6: facc=0.915413
2018-09-16 03:37:00,428 INFO - LoggingTest - 
bina

2018-09-16 03:37:37,820:304:INFO:speed: 89.8316434677 samples/s
2018-09-16 03:37:37,820 INFO - LoggingTest - speed: 89.8316434677 samples/s
2018-09-16 03:37:38,030:307:INFO:discriminator loss = 0.366773, generator loss = 26.471222, binary training acc = 0.805207 at iter 10 epoch 10
2018-09-16 03:37:38,030 INFO - LoggingTest - discriminator loss = 0.366773, generator loss = 26.471222, binary training acc = 0.805207 at iter 10 epoch 10
2018-09-16 03:37:41,320:304:INFO:speed: 91.4058072478 samples/s
2018-09-16 03:37:41,320 INFO - LoggingTest - speed: 91.4058072478 samples/s
2018-09-16 03:37:41,530:307:INFO:discriminator loss = 0.229258, generator loss = 28.166958, binary training acc = 0.832175 at iter 20 epoch 10
2018-09-16 03:37:41,530 INFO - LoggingTest - discriminator loss = 0.229258, generator loss = 28.166958, binary training acc = 0.832175 at iter 20 epoch 10
2018-09-16 03:37:44,824:304:INFO:speed: 91.8516251553 samples/s
2018-09-16 03:37:44,824 INFO - LoggingTest - speed: 91.85162

...

2018-09-16 04:07:27,287:304:INFO:speed: 91.8808636482 samples/s
2018-09-16 04:07:27,287 INFO - LoggingTest - speed: 91.8808636482 samples/s
2018-09-16 04:07:27,497:307:INFO:discriminator loss = 0.447200, generator loss = 9.865664, binary training acc = 0.655222 at iter 20 epoch 169
2018-09-16 04:07:27,497 INFO - LoggingTest - discriminator loss = 0.447200, generator loss = 9.865664, binary training acc = 0.655222 at iter 20 epoch 169
2018-09-16 04:07:30,802:304:INFO:speed: 92.4054590506 samples/s
2018-09-16 04:07:30,802 INFO - LoggingTest - speed: 92.4054590506 samples/s
2018-09-16 04:07:31,012:307:INFO:discriminator loss = 0.444616, generator loss = 10.107221, binary training acc = 0.667170 at iter 30 epoch 169
2018-09-16 04:07:31,012 INFO - LoggingTest - discriminator loss = 0.444616, generator loss = 10.107221, binary training acc = 0.667170 at iter 30 epoch 169
2018-09-16 04:07:31,161:313:INFO:
binary training acc at epoch 169: facc=0.669914
2018-09-16 04:07:31,161 INFO - LoggingTe

2018-09-16 04:08:12,161:304:INFO:speed: 91.2348783888 samples/s
2018-09-16 04:08:12,161 INFO - LoggingTest - speed: 91.2348783888 samples/s
2018-09-16 04:08:12,371:307:INFO:discriminator loss = 0.517873, generator loss = 10.521414, binary training acc = 0.669575 at iter 20 epoch 173
2018-09-16 04:08:12,371 INFO - LoggingTest - discriminator loss = 0.517873, generator loss = 10.521414, binary training acc = 0.669575 at iter 20 epoch 173
2018-09-16 04:08:15,667:304:INFO:speed: 93.3685295008 samples/s
2018-09-16 04:08:15,667 INFO - LoggingTest - speed: 93.3685295008 samples/s
2018-09-16 04:08:15,876:307:INFO:discriminator loss = 0.432130, generator loss = 10.031671, binary training acc = 0.658170 at iter 30 epoch 173
2018-09-16 04:08:15,876 INFO - LoggingTest - discriminator loss = 0.432130, generator loss = 10.031671, binary training acc = 0.658170 at iter 30 epoch 173
2018-09-16 04:08:16,027:313:INFO:
binary training acc at epoch 173: facc=0.655388
2018-09-16 04:08:16,027 INFO - Logging

2018-09-16 04:08:53,465:304:INFO:speed: 92.831432915 samples/s
2018-09-16 04:08:53,465 INFO - LoggingTest - speed: 92.831432915 samples/s
2018-09-16 04:08:53,675:307:INFO:discriminator loss = 0.642292, generator loss = 9.710346, binary training acc = 0.559643 at iter 10 epoch 177
2018-09-16 04:08:53,675 INFO - LoggingTest - discriminator loss = 0.642292, generator loss = 9.710346, binary training acc = 0.559643 at iter 10 epoch 177
2018-09-16 04:08:56,963:304:INFO:speed: 91.8234103718 samples/s
2018-09-16 04:08:56,963 INFO - LoggingTest - speed: 91.8234103718 samples/s
2018-09-16 04:08:57,175:307:INFO:discriminator loss = 0.518997, generator loss = 10.192595, binary training acc = 0.606200 at iter 20 epoch 177
2018-09-16 04:08:57,175 INFO - LoggingTest - discriminator loss = 0.518997, generator loss = 10.192595, binary training acc = 0.606200 at iter 20 epoch 177
2018-09-16 04:09:00,463:304:INFO:speed: 92.5428250517 samples/s
2018-09-16 04:09:00,463 INFO - LoggingTest - speed: 92.54282

2018-09-16 04:09:38,263:304:INFO:speed: 91.4619910458 samples/s
2018-09-16 04:09:38,263 INFO - LoggingTest - speed: 91.4619910458 samples/s
2018-09-16 04:09:38,475:307:INFO:discriminator loss = 0.588434, generator loss = 9.349669, binary training acc = 0.595844 at iter 10 epoch 181
2018-09-16 04:09:38,475 INFO - LoggingTest - discriminator loss = 0.588434, generator loss = 9.349669, binary training acc = 0.595844 at iter 10 epoch 181
2018-09-16 04:09:41,760:304:INFO:speed: 90.8288069297 samples/s
2018-09-16 04:09:41,760 INFO - LoggingTest - speed: 90.8288069297 samples/s
2018-09-16 04:09:41,971:307:INFO:discriminator loss = 0.522443, generator loss = 10.682011, binary training acc = 0.655471 at iter 20 epoch 181
2018-09-16 04:09:41,971 INFO - LoggingTest - discriminator loss = 0.522443, generator loss = 10.682011, binary training acc = 0.655471 at iter 20 epoch 181
2018-09-16 04:09:45,262:304:INFO:speed: 91.6448473311 samples/s
2018-09-16 04:09:45,262 INFO - LoggingTest - speed: 91.644

2018-09-16 04:10:26,615:304:INFO:speed: 90.9948847562 samples/s
2018-09-16 04:10:26,615 INFO - LoggingTest - speed: 90.9948847562 samples/s
2018-09-16 04:10:26,823:307:INFO:discriminator loss = 0.508111, generator loss = 9.902016, binary training acc = 0.625417 at iter 20 epoch 185
2018-09-16 04:10:26,823 INFO - LoggingTest - discriminator loss = 0.508111, generator loss = 9.902016, binary training acc = 0.625417 at iter 20 epoch 185
2018-09-16 04:10:30,105:304:INFO:speed: 92.9991865378 samples/s
2018-09-16 04:10:30,105 INFO - LoggingTest - speed: 92.9991865378 samples/s
2018-09-16 04:10:30,315:307:INFO:discriminator loss = 0.582727, generator loss = 10.221522, binary training acc = 0.655152 at iter 30 epoch 185
2018-09-16 04:10:30,315 INFO - LoggingTest - discriminator loss = 0.582727, generator loss = 10.221522, binary training acc = 0.655152 at iter 30 epoch 185
2018-09-16 04:10:30,464:313:INFO:
binary training acc at epoch 185: facc=0.654437
2018-09-16 04:10:30,464 INFO - LoggingTe

In [12]:
predictor = mxnet_estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

INFO:sagemaker:Creating model with name: sagemaker-mxnet-2018-09-16-03-31-05-846


ClientError: An error occurred (ValidationException) when calling the CreateModel operation: Could not find model data at s3://sagemaker-ap-northeast-1-373011628954/sagemaker-mxnet-2018-09-16-03-31-05-846/output/model.tar.gz.